#**Titanic - Machine Learning from Disaster**

*   Utilizaremos os datasets disponíveis no [Kaggle Competitions](https://www.kaggle.com/competitions/titanic/overview);
*   A pontuação é a porcentagem de sobreviventes previstos corretamente (Acurácia);

*   **Parte 1** disponível [aqui](https://colab.research.google.com/drive/1MF4oDo24VNhvFihfC0AXUbMUvDvsXCCr#scrollTo=Hn82KfnIwYw6)
    * Resultado da submissão: **0.66746**
*   **Parte 2** disponível [aqui](https://colab.research.google.com/drive/1k_bztwNR3HsEU3SkisVqKupLrgmBL_wn#scrollTo=FLEmuuJj_I7F)
    * Resultado da submissão: **0.76555**
*   **Parte 3** disponível [aqui](https://colab.research.google.com/drive/1UdXAHPYIqxHDdvEX1eczT_xji0_vWrZm#scrollTo=s--bRTFUNasn)
    * Resultado da submissão: **0.77751**

# Parte 1, 2 e 3

Repetiremos os iniciais passos realizados na **Parte 1**, na **Parte 2** e na **Parte 3**:

In [ ]:
#Importando pandas
import pandas as pd
treino = pd.read_csv("train.csv")
teste = pd.read_csv("test.csv")

In [ ]:
#Eliminando colunas com alta cardinalidade
treino = treino.drop(['Name','Ticket','Cabin'], axis = 1)
teste = teste.drop(['Name','Ticket','Cabin'], axis = 1)

In [ ]:
#Substituindo idades vazias pela média das idades
treino.loc[treino.Age.isnull(), 'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(), 'Age'] = teste.Age.mean()

In [ ]:
#Substituindo vazios pela moda
treino.loc[treino.Embarked.isnull(), 'Embarked'] = treino.Embarked.mode()[0]

In [ ]:
#Substituindo vazios pela média da Fare
teste.loc[teste.Fare.isnull(), 'Fare'] = teste.Fare.mean()

In [ ]:
#Usando lambda para criação da nova coluna com valores que diferenciam o gênero
treino['Genero'] = treino.Sex.apply(lambda x:1 if x == 'male' else 0)
teste['Genero'] = teste.Sex.apply(lambda x:1 if x == 'male' else 0)

In [ ]:
#Importação
from sklearn.preprocessing import RobustScaler

#Criar o Scaler / redimensionador
transformer = RobustScaler().fit(treino[['Age','Fare']])

#Realizando a transformação na base treino e atualizando as informações
treino[['Age','Fare']] = transformer.transform(treino[['Age','Fare']])

#Criando transformador e realizando a transformação na base teste e atualizando as informações
transformer = RobustScaler().fit(teste[['Age','Fare']])
teste[['Age','Fare']] = transformer.transform(teste[['Age','Fare']])

In [ ]:
#Função de identificação dos que viajam sozinhos
def sozinho (a,b):
    if (a ==0 and b ==0):
        return 1
    else:
        return 0

#Aplicando função
treino['Sozinho'] = treino.apply(lambda x: sozinho(x.SibSp,x.Parch),axis=1)
teste['Sozinho'] = teste.apply(lambda x: sozinho(x.SibSp,x.Parch),axis=1)

In [ ]:
#Somando os familiares a bordo
treino['Familiares'] = treino.SibSp + treino.Parch
teste['Familiares'] = teste.SibSp + teste.Parch

In [ ]:
#Ordenar de acordo com o local de embarque, usando o OrdinalEncoder:
#Importando
from sklearn.preprocessing import OrdinalEncoder
#Criando encoder
categorias = ['S','C','Q']
enc = OrdinalEncoder(categories = [categorias],dtype='int32')
#Fit com os dados
enc.fit(treino[['Embarked']])
#Adicionando coluna na base original
treino['Embarked'] = enc.transform(treino[['Embarked']])
teste['Embarked'] = enc.transform(teste[['Embarked']])

In [ ]:
#Eliminando colunas desnecessárias
treino = treino.drop(['Sex'],axis=1)
teste = teste.drop(['Sex'],axis=1)

In [ ]:
#Visualizando a base
treino.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Genero,Sozinho,Familiares
0,1,0,3,-0.592240,1,0,-0.312011,0,1,0,1
1,2,1,1,0.638529,1,0,2.461242,1,0,0,1
2,3,1,3,-0.284548,0,0,-0.282777,0,0,1,0
3,4,1,1,0.407760,1,0,1.673732,0,0,0,1
4,5,0,3,0.407760,0,0,-0.277363,0,1,1,0


# Parte 4
# **Testando Novos Classificadores e Melhorando Parâmetros**



**Primeira classificação, antes de realizar alterações:**
*   Faremos uma classificação inicial como ponto de partida, usando os novos modelos de classificação.
*   O modelo de classificação por regressão logística foi mantido por apresentar melhor resultado nas modelagens anteriores.

**Criação de modelos para classificação:**
*   1 - Precisamos dividir a base de treino em treino e validação com o [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html);
*   2 - Vamos usar os seguintes classificadores:
    * [Regressão Logística](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html);
    * [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html);
    * [MLP Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html);

In [ ]:
#Importar train_test_split
from sklearn.model_selection import train_test_split

#Separando base de treino em X e Y
X = treino.drop(['PassengerId','Survived'],axis=1)
y = treino.Survived

#Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.33,random_state=42)

In [ ]:
#Regressão logística

#Importação
from sklearn.linear_model import LogisticRegression

#Criando cassificador
clf_rl = LogisticRegression(random_state=42,max_iter=1000)

#Fit com os dados
clf_rl = clf_rl.fit(X_train, y_train)

#Realizando previsão
y_pred_rl = clf_rl.predict(X_val)

In [ ]:
#Random Forest

#importação
from sklearn.ensemble import RandomForestClassifier

#Criando cassificador
clf_rf = RandomForestClassifier(random_state=42)

#Fit com os dados
clf_rf = clf_rf.fit(X_train, y_train)

#Realizando previsão
y_pred_rf = clf_rf.predict(X_val)

In [ ]:
#MLP Classifier

#importação
from sklearn.neural_network import MLPClassifier

#Criando cassificador
clf_mlp = MLPClassifier(random_state=42, max_iter=5000)

#Fit com os dados
clf_mlp = clf_mlp.fit(X_train, y_train)

#Realizando previsão
y_pred_mlp = clf_mlp.predict(X_val)

**Avaliando os modelos por:**
*   1 - [Acurácia](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
*   2 - [Matriz de confusão](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

In [ ]:
#Importação Acurácia:
from sklearn.metrics import accuracy_score

In [ ]:
#Regressão Logística
acc_scr_rl = accuracy_score(y_val, y_pred_rl)
acc_scr_rl

0.8067796610169492

In [ ]:
#Random Forest
acc_scr_rf = accuracy_score(y_val, y_pred_rf)
acc_scr_rf

0.7898305084745763

In [ ]:
#MLP Classifier
acc_scr_mlp = accuracy_score(y_val, y_pred_mlp)
acc_scr_mlp

0.8305084745762712

**Criando DF para salvar os valores de acurácia para posteior comparação:**

In [ ]:
acc_scr_df = pd.DataFrame({
    'modelos': ['Reg. Log.','Random','MLP Classifier'],
    'resultados iniciais': [acc_scr_rl,acc_scr_rf,acc_scr_mlp]
})
acc_scr_df

,modelos,resultados iniciais
0,Reg. Log.,0.806780
1,Random,0.789831
2,MLP Classifier,0.830508


In [ ]:
#Importação Matriz de Confusão
from sklearn.metrics import confusion_matrix

In [ ]:
#Regressão Logística
confusion_matrix(y_val, y_pred_rl)

array([[152,  23],
       [ 34,  86]])

In [ ]:
#Random Forest
confusion_matrix(y_val, y_pred_rf)

array([[148,  27],
       [ 35,  85]])

In [ ]:
#MLP Classifier
confusion_matrix(y_val, y_pred_mlp)

array([[160,  15],
       [ 35,  85]])

**Fazendo previsão na base teste:**
*   Usaremos o modelo com melhor precisão para fazer a previsão (predict)

In [ ]:
#Visualizando o X_train
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Embarked,Genero,Sozinho,Familiares
6,1,1.869299,0,0,1.620136,0,1,1,0
718,3,0.000000,0,0,0.045293,2,1,1,0
685,2,-0.361471,1,2,1.174771,1,1,0,3
73,3,-0.284548,1,0,0.000000,1,1,0,1
882,3,-0.592240,0,0,-0.170531,0,0,1,0


In [ ]:
##Visualizando a base teste
teste.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked,Genero,Sozinho,Familiares
0,892,3,0.331562,0,0,-0.280670,2,1,1,0
1,893,3,1.311954,1,0,-0.315800,0,0,0,1
2,894,2,2.488424,0,0,-0.201943,2,1,1,0
3,895,3,-0.256674,0,0,-0.245367,0,1,1,0
4,896,3,-0.648831,1,1,-0.091793,0,0,0,2


In [ ]:
#Eliminar coluna ID para que a base teste seja igual a base treino
X_teste = teste.drop(['PassengerId'], axis=1)

In [ ]:
#Aplicando MLP
y_pred = clf_mlp.predict(X_teste)

In [ ]:
#Criando coluna com a previsão na base teste
teste['Survived'] = y_pred

In [ ]:
#Selecionando colunas de interesse para fazer envio ao Kaggle
envio_kaggle = teste[['PassengerId','Survived']]

In [ ]:
#Exportando .csv
envio_kaggle.to_csv('resultados_parte04_1.csv',index=False)

Resultado da submissão: **0.74880**

**Criação de modelos para classificação:**
*   1 - Precisamos dividir a base de treino em treino e validação com o [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html);
*   2 - Vamos usar os seguintes classificadores:
    * [Regressão Logística](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html);
    * [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html);
    * [MLP Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html);
*   3 - Estimar os melhores parâmetros com o [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) e evitar Overfitting com o [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html);
    * Definiremos os parâmetros que desejamos analisar para escolher o melhor dentre eles, não podemos deixar isso em aberto, pois os modelos possuem muitos parâmetros e isso tomaria muito tempo.

In [ ]:
#Importar train_test_split
from sklearn.model_selection import train_test_split

#Separando base de treino em X e Y
X = treino.drop(['PassengerId','Survived'],axis=1)
y = treino.Survived

#Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
#Regressão Logística:


#importação
from sklearn.linear_model import LogisticRegression

#Criando cassificador
clf_rl = LogisticRegression(random_state=42)

#Definindo parâmetros
parametros_rl = {
    'penalty' : ['l1','l2'],
    'C' : [0.01,0.1,1,10],
    'solver' : ['lbfgs','liblinear','saga'],
    'max_iter' : [100,1000,5000,10000]
}

In [ ]:
#Random Forest:


#importação
from sklearn.ensemble import RandomForestClassifier

#Criando cassificador
clf_rf = RandomForestClassifier(random_state=42)

#Definindo parâmetros
parametros_rf = {
    'n_estimators':[100,200,500,1000],
    'criterion':['gini','entropy','log_loss'],
    'max_depth':[2,4,6,8,None],
    'max_features':['sqrt','log2', None]
}

In [ ]:
#MLP Classifier


#importação
from sklearn.neural_network import MLPClassifier

#Criando cassificador
clf_mlp = MLPClassifier(random_state=42)

#Definindo parâmetros
parametros_mlp = {
    'solver':['lbfgs','sgd','adam'],
    'alpha': [10.0**(-1),10.0**(-5),10.0**(-7),10.0**(-10)],
    'max_iter':[100,500,1000,5000]
}

In [ ]:
#Importar GridSearchCV
from sklearn.model_selection import GridSearchCV
#Importar KFold
from sklearn.model_selection import KFold

#Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

#Usando DateTime para verificar o tempo de execução de cada classificador
from datetime import datetime
def hora_atual():
    agora = datetime.now()
    print(str(agora.hour)+':'+str(agora.minute)+':'+str(agora.second))

In [ ]:
##Regressão Logística:

hora_atual()
kfold_rl = KFold(shuffle=True,random_state=42,n_splits=8)
grid_rl = GridSearchCV(clf_rl,parametros_rl,scoring='accuracy',cv=kfold_rl)
grid_rl = grid_rl.fit(X_train,y_train)
hora_atual()

20:39:57
20:40:9


In [ ]:
#Random Forest:

hora_atual()
kfold_rf = KFold(shuffle=True,random_state=42,n_splits=8)
grid_rf = GridSearchCV(clf_rf,parametros_rf,scoring='accuracy',cv=kfold_rf)
grid_rf = grid_rf.fit(X_train,y_train)
hora_atual()

20:40:23
21:2:40


In [ ]:
#MLP Classifier

hora_atual()
kfold_mlp = KFold(shuffle=True,random_state=42,n_splits=8)
grid_mlp = GridSearchCV(clf_mlp,parametros_mlp,scoring='accuracy',cv=kfold_mlp)
grid_mlp = grid_mlp.fit(X_train,y_train)
hora_atual()

21:2:41
21:15:43


**Verificando os melhores scores:**
*   Identificando a melhor pontuação usando os melhores parâmetros encontrados(dentre os que definimos) em cada modelo de classificação.

In [ ]:
#Regressão Logística:

grid_rl.best_score_

0.8089887640449438

In [ ]:
#Random Forest:

grid_rf.best_score_

0.8314606741573034

In [ ]:
#MLP Classifier

grid_mlp.best_score_

0.8188202247191012

**Verificando os melhores parâmetros:**
*   Identificando os melhores parâmetros encontrados(dentre os que definimos) em cada modelo de classificação, responsáveis pelo escore acima.

In [ ]:
#Regressão Logística:

grid_rl.best_params_

{'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}

In [ ]:
#Random Forest:

grid_rf.best_params_

{'criterion': 'entropy',
 'max_depth': 6,
 'max_features': 'sqrt',
 'n_estimators': 100}

In [ ]:
#MLP Classifier

grid_mlp.best_params_

{'alpha': 1e-05, 'max_iter': 100, 'solver': 'adam'}

**Fazendo previsão com os melhores modelos:**
*   Aqui faremos a previsão usando os classificadores com os parâmetros com melhor resultado

In [ ]:
#Regressão Logística:

clf_best_rl = grid_rl.best_estimator_
y_pred_rl = clf_best_rl.predict(X_val)

In [ ]:
#Random Forest:

clf_best_rf = grid_rf.best_estimator_
y_pred_rf = clf_best_rf.predict(X_val)

In [ ]:
#MLP Classifier

clf_best_mlp = grid_mlp.best_estimator_
y_pred_mlp = clf_best_mlp.predict(X_val)

**Avaliando os modelos por:**
*   1 - [Acurácia](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
*   2 - [Matriz de confusão](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

In [ ]:
#Importação Acurácia:
from sklearn.metrics import accuracy_score

In [ ]:
#Regressão Logística
acc_scr_rl = accuracy_score(y_val, y_pred_rl)
acc_scr_rl

0.8044692737430168

In [ ]:
#Random Forest
acc_scr_rf = accuracy_score(y_val, y_pred_rf)
acc_scr_rf

0.8100558659217877

In [ ]:
#MLP Classifier
acc_scr_mlp = accuracy_score(y_val, y_pred_mlp)
acc_scr_mlp

0.7988826815642458

In [ ]:
#Importação Matriz de Confusão
from sklearn.metrics import confusion_matrix

In [ ]:
#Regressão Logística
confusion_matrix(y_val, y_pred_rl)

array([[91, 14],
       [21, 53]])

In [ ]:
#Random Forest
confusion_matrix(y_val, y_pred_rf)

array([[93, 12],
       [22, 52]])

In [ ]:
#MLP Classifier
confusion_matrix(y_val, y_pred_mlp)

array([[91, 14],
       [22, 52]])

**Fazendo previsão na base teste:**
*   Usaremos o modelo com melhor precisão para fazer a previsão (predict)

In [ ]:
#Visualizando o X_train
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Embarked,Genero,Sozinho,Familiares
331,1,1.215452,0,0,0.608317,0,1,1,0
733,2,-0.515317,0,0,-0.062981,0,1,1,0
382,3,0.176991,0,0,-0.282777,0,1,1,0
704,3,-0.284548,1,0,-0.285843,0,1,0,1
813,3,-1.823009,4,2,0.728501,0,0,0,6


In [ ]:
##Visualizando a base teste
teste.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked,Genero,Sozinho,Familiares,Survived
0,892,3,0.331562,0,0,-0.280670,2,1,1,0,0
1,893,3,1.311954,1,0,-0.315800,0,0,0,1,0
2,894,2,2.488424,0,0,-0.201943,2,1,1,0,0
3,895,3,-0.256674,0,0,-0.245367,0,1,1,0,0
4,896,3,-0.648831,1,1,-0.091793,0,0,0,2,0


In [ ]:
#Eliminar coluna ID para que a base teste seja igual a base treino
X_teste = teste.drop(['PassengerId','Survived'], axis=1)

In [ ]:
#Aplicando RF
y_pred = clf_best_rf.predict(X_teste)

In [ ]:
#Criando coluna com a previsão na base teste
teste['Survived'] = y_pred

In [ ]:
#Selecionando colunas de interesse para fazer envio ao Kaggle
envio_kaggle = teste[['PassengerId','Survived']]

In [ ]:
#Exportando .csv
envio_kaggle.to_csv('resultados_parte04_2.csv',index=False)

Resultado da submissão: **0.77990**